# kpx data
- 거래시간 1H는 00:01~01:00 을 의미함
- 풍력 전력거래량\n(단위: MWh)
- 태양광 전력거래량\n(단위: MWh)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime
import pickle
import math

In [2]:
df_wind = pd.read_excel(os.path.abspath(os.path.join(os.getcwd(), '..', 'data', 'raw', 'kpx', 'wind', '풍력_전력거래량_서귀포시 표선면 가시리_20170701~20190731.xlsx')))
df_solar = pd.read_excel(os.path.abspath(os.path.join(os.getcwd(), '..', 'data', 'raw', 'kpx', 'solar', '태양광_전력거래량_전남 해남군 황산면_20170701~20190731.xlsx')))
display(df_wind[0:10])
display(df_solar[0:10])

,Unnamed: 0,Unnamed: 1,Unnamed: 2
0,○ 주소 : 제주특별자치도 서귀포시 표선면 가시리,NaN,NaN
1,"○ 총 설비용량(MW, 발전단 기준) : 45",NaN,NaN
2,※ 전력거래량은 송전단 기준의 발전량임,NaN,NaN
3,※ 거래시간 1H는 00:01~01:00 을 의미함,NaN,NaN
4,NaN,NaN,NaN
5,거래일,시간,풍력 전력거래량\n(단위: MWh)
6,2017-07-01 00:00:00,1,1.09776
7,2017-07-01 00:00:00,2,1.386
8,2017-07-01 00:00:00,3,0.546
9,2017-07-01 00:00:00,4,0


,Unnamed: 0,Unnamed: 1,Unnamed: 2
0,○ 주소 : 전라남도 해남군 황산면,NaN,NaN
1,"○ 총 설비용량(MW, 발전단 기준) : 7.381",NaN,NaN
2,※ 전력거래량은 송전단 기준의 발전량임,NaN,NaN
3,※ 거래시간 1H는 00:01~01:00 을 의미함,NaN,NaN
4,NaN,NaN,NaN
5,거래일,시간,태양광 전력거래량\n(단위: MWh)
6,2017-07-01 00:00:00,1,0
7,2017-07-01 00:00:00,2,0
8,2017-07-01 00:00:00,3,0
9,2017-07-01 00:00:00,4,0


In [3]:
def fix_hour(x) :
    x = int(x) - 1
    x = str(x)
    if len(x) == 1 :
        x= '0' + x
    x = x + ':00:00'
    return x

def preprocess(df, location, generation_type) :
    df = df.drop(df.index[[0, 1, 2, 3, 4, 5]])
    df.columns = ['date', 'hour', '발전량(kW)']
    df['발전량(kW)'] = df['발전량(kW)'] * 1000
    
    df['hour'] = df['hour'].apply(fix_hour)

    df['date'] = df['date'].astype(str)
    df['datetime'] = df['date'] + ' ' + df['hour']
    df['datetime'] = pd.to_datetime(df['datetime'])
    df['date'] = pd.to_datetime(df['date'])
    
    df = df.drop(['hour'], axis=1)

    df['location'] = np.empty(df.shape[0])
    df['location'] = df['location'].apply(lambda x : location)
    
    if generation_type == 'wind' :
        df['풍속(m/s)'] = np.empty(df.shape[0])
        df['풍속(m/s)'] = df['풍속(m/s)'].apply(lambda x : 'NaN')
        df['풍향(16방위)'] = np.empty(df.shape[0])
        df['풍향(16방위)'] = df['풍향(16방위)'].apply(lambda x : 'NaN')
        
    if generation_type == 'solar' :
        df['일사량(MJ/m^2)'] = np.empty(df.shape[0])
        df['일사량(MJ/m^2)'] = df['일사량(MJ/m^2)'].apply(lambda x : 'NaN')

    return df

In [4]:
def show_all_days(df, height, width) : 
    print(df['date'].unique().shape[0])
    
    b = 10
    a = math.ceil(df['date'].unique().shape[0]/b)
    n = 0
    
    fig, axs = plt.subplots(a,b)
    fig.set_figheight(height)
    fig.set_figwidth(width)
    
    for i in range(a) :
        for j in range(b) :
            try :
                axs[i, j].plot(df[df['date']==df['date'].unique()[b*i + j]]['발전량(kW)'])
                #axs[i, j].set_title(df['date'].unique()[b*i + j])
            except Exception as e:
                n += 1
                
    plt.tight_layout()
    plt.show()
    return

## 풍력

In [6]:
location = 'Korea, Jeju-do, Seogwipo-si, Pyoseon-myeon'
df_wind = preprocess(df_wind, location, 'wind')
display(df_wind.head())
display(df_wind.tail())

,date,발전량(kW),datetime,location,풍속(m/s),풍향(16방위)
6,2017-07-01,1097.76,2017-07-01 00:00:00,"Korea, Jeju-do, Seogwipo-si, Pyoseon-myeon",NaN,NaN
7,2017-07-01,1386,2017-07-01 01:00:00,"Korea, Jeju-do, Seogwipo-si, Pyoseon-myeon",NaN,NaN
8,2017-07-01,546,2017-07-01 02:00:00,"Korea, Jeju-do, Seogwipo-si, Pyoseon-myeon",NaN,NaN
9,2017-07-01,0,2017-07-01 03:00:00,"Korea, Jeju-do, Seogwipo-si, Pyoseon-myeon",NaN,NaN
10,2017-07-01,7168.32,2017-07-01 04:00:00,"Korea, Jeju-do, Seogwipo-si, Pyoseon-myeon",NaN,NaN


,date,발전량(kW),datetime,location,풍속(m/s),풍향(16방위)
18265,2019-07-31,11005.4,2019-07-31 19:00:00,"Korea, Jeju-do, Seogwipo-si, Pyoseon-myeon",NaN,NaN
18266,2019-07-31,6233.04,2019-07-31 20:00:00,"Korea, Jeju-do, Seogwipo-si, Pyoseon-myeon",NaN,NaN
18267,2019-07-31,719.52,2019-07-31 21:00:00,"Korea, Jeju-do, Seogwipo-si, Pyoseon-myeon",NaN,NaN
18268,2019-07-31,1153.92,2019-07-31 22:00:00,"Korea, Jeju-do, Seogwipo-si, Pyoseon-myeon",NaN,NaN
18269,2019-07-31,1410.96,2019-07-31 23:00:00,"Korea, Jeju-do, Seogwipo-si, Pyoseon-myeon",NaN,NaN


In [ ]:
show_all_days(df_wind, 200, 20)

## 태양광

In [7]:
location = 'Korea, Jeollanam-do, Haenam-gun, Haenam-eup'
df_solar = preprocess(df_solar, location, 'solar')
display(df_solar.head())
display(df_solar.tail())

,date,발전량(kW),datetime,location,일사량(MJ/m^2)
6,2017-07-01,0,2017-07-01 00:00:00,"Korea, Jeollanam-do, Haenam-gun, Haenam-eup",NaN
7,2017-07-01,0,2017-07-01 01:00:00,"Korea, Jeollanam-do, Haenam-gun, Haenam-eup",NaN
8,2017-07-01,0,2017-07-01 02:00:00,"Korea, Jeollanam-do, Haenam-gun, Haenam-eup",NaN
9,2017-07-01,0,2017-07-01 03:00:00,"Korea, Jeollanam-do, Haenam-gun, Haenam-eup",NaN
10,2017-07-01,0,2017-07-01 04:00:00,"Korea, Jeollanam-do, Haenam-gun, Haenam-eup",NaN


,date,발전량(kW),datetime,location,일사량(MJ/m^2)
18265,2019-07-31,6.421,2019-07-31 19:00:00,"Korea, Jeollanam-do, Haenam-gun, Haenam-eup",NaN
18266,2019-07-31,0,2019-07-31 20:00:00,"Korea, Jeollanam-do, Haenam-gun, Haenam-eup",NaN
18267,2019-07-31,0,2019-07-31 21:00:00,"Korea, Jeollanam-do, Haenam-gun, Haenam-eup",NaN
18268,2019-07-31,0,2019-07-31 22:00:00,"Korea, Jeollanam-do, Haenam-gun, Haenam-eup",NaN
18269,2019-07-31,0,2019-07-31 23:00:00,"Korea, Jeollanam-do, Haenam-gun, Haenam-eup",NaN


In [ ]:
show_all_days(df_solar, 200, 20)

## save

In [ ]:
df_wind.to_pickle(os.path.abspath(os.path.join(os.getcwd(), '..', 'data', 'df_kpx_wind.pkl')))

In [ ]:
df_solar.to_pickle(os.path.abspath(os.path.join(os.getcwd(), '..', 'data', 'df_kpx_solar.pkl')))